In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

# DATA QUERIES

In [ ]:
allfiles_sim = glob.glob(r'G:\My Drive\DOCUMENTS\PROVISIONES\2021\04. ABRIL\*.csv', recursive=True)

In [ ]:
sim_all = pd.concat([pd.read_csv(f,dtype={'AMOUNT':float,'CLAVE_AUTORIZACION': object,'BIN':object,'LAST4':object,'ORDER_ID':object,'DEVICE_ID':object})
                              .assign(file_name_tableros=os.path.basename(f)) for f in allfiles_sim])
sim_all.columns = sim_all.columns.str.strip().str.lower()
sim_all['msi'].fillna(0,inplace=True)
sim_all.head()

In [ ]:
sim_all.shape

In [ ]:
all_provisiones = sim_all[(sim_all['status_code'] == 1)]
all_provisiones.head()

In [ ]:
all_provisiones.shape

In [ ]:
all_provisiones['psp'].unique()

In [ ]:
all_provisiones.dtypes

# PROVISIONES BANORTE

In [ ]:
provision_bnte = all_provisiones[(all_provisiones['psp'] == 'PROSA')]
provision_bnte['CONCAT'] = provision_bnte['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_bnte['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_bnte['last4'].astype(str).replace(' ', '',regex=True)
provision_bnte.head()

In [ ]:
provision_bnte.shape

### MAYO 2021

In [ ]:
name_file=["Tipo de Columna","Afiliación","Referencia","Tarjeta","Monto depositado","Comisiones","Fecha txn","Importe Bruto"
           ,"Tipo Moneda","Tipo txn","Iva","Iva por pago diferido","Iva pagado por cashback","Iva cobrado por cashback"
           ,"Comisión pagada por cashback","Comisión cobrada por cashback","Comisión por pago diferido","Monto del Cashback"
           ,"Núm Pagos","Diferimiento Inicial","Tipo de Plan","Código de Autorización","Empty","ECI","Ref. Interbancaria"
           ,"Empty.1","Empty.2","Empty.3","order_id_xpay","Ref. de Payworks","Customer_ref1","Customer_ref2","Customer_ref3"
           ,"Núm de Control"]
dtypes_file={'Importe Bruto':float,'Ref. Interbancaria': object,'Afiliación':object,'Referencia':object,'Tarjeta':object
             ,'Núm Pagos':object,'Diferimiento Inicial':object,'Tipo de Plan':object,'Código de Autorización':object
             ,'ECI':object,'order_id_xpay':object}

In [ ]:
use_cols = ['Afiliación','Tipo txn','Fecha txn','Importe Bruto','Código de Autorización','Ref. Interbancaria','order_id_xpay','Tarjeta']

In [ ]:
sftp_may_1_21 = glob.glob(r'G:\My Drive\2.-CONCILIACIONES Y POLIZA\SFTP BANORTE\2021 SFTP Banorte\05. MAYO\**\PAYCLIP_Settlement*.csv', recursive=False)
sftp_may_1_21

In [ ]:
sftp_may_21 = pd.concat([pd.read_csv(f,names=name_file,dtype=dtypes_file,usecols=use_cols)
                         .assign(file_name=os.path.basename(f)) for f in sftp_may_1_21])
sftp_may_21.rename(columns ={'Ref. Interbancaria':'order_id','Código de Autorización':'codigo_autorizacion'}, inplace=True)
sftp_may_21['date_file'] = sftp_may_21["file_name"].str[-15:-7]
sftp_may_21['date_file']= pd.to_datetime(sftp_may_21['date_file'], format='%d%m%Y').dt.date
sftp_may_21['date_file']= pd.to_datetime(sftp_may_21['date_file'])
sftp_may_21.sort_values(by=['date_file'],ascending=True,inplace=True)
sftp_may_21['month'] = sftp_may_21['date_file'].dt.strftime('%B')
sftp_may_21.head()

In [ ]:
sftp_may_21.shape

### ABRIL 2021

In [ ]:
sftp_apr_1_21 = glob.glob(r'G:\My Drive\2.-CONCILIACIONES Y POLIZA\SFTP BANORTE\2021 SFTP Banorte\04. ABRIL\**\PAYCLIP_Settlement*.csv', recursive=False)
sftp_apr_1_21

In [ ]:
sftp_apr_21 = pd.concat([pd.read_csv(f,names=name_file,dtype=dtypes_file,usecols=use_cols)
                         .assign(file_name=os.path.basename(f)) for f in sftp_apr_1_21])
sftp_apr_21.rename(columns ={'Ref. Interbancaria':'order_id','Código de Autorización':'codigo_autorizacion'}, inplace=True)
sftp_apr_21['date_file'] = sftp_apr_21["file_name"].str[-15:-7]
sftp_apr_21['date_file']= pd.to_datetime(sftp_apr_21['date_file'], format='%d%m%Y').dt.date
sftp_apr_21['date_file']= pd.to_datetime(sftp_apr_21['date_file'])
sftp_apr_21.sort_values(by=['date_file'],ascending=True,inplace=True)
sftp_apr_21['month'] = sftp_apr_21['date_file'].dt.strftime('%B')
sftp_apr_21.head()

In [ ]:
sftp_apr_21.shape

### SFTP CONCAT

In [ ]:
sftp = pd.concat([sftp_apr_21,sftp_may_21])
sftp.head()

In [ ]:
# sftp.drop(['Empty','Empty.1','Empty.2','Empty.3','Empty.4'],axis=1,inplace=True)
sftp.rename(columns ={'Ref. Interbancaria':'order_id'}, inplace=True)
sftp.head()

In [ ]:
sftp.shape

In [ ]:
sftp_fecha_deposito = sftp[(sftp['Tipo txn'] == 'BANORTE')]
sftp_fecha_deposito = sftp_fecha_deposito[['date_file', 'Importe Bruto']]
sftp_fecha_deposito.rename(columns ={'Importe Bruto': 'Fecha_deposito'},inplace=True)                                                
sftp_fecha_deposito.head()

In [ ]:
sftp = sftp[(sftp['Tipo txn'] == 'SETTLED')]
sftp = sftp.merge(sftp_fecha_deposito[['date_file', 'Fecha_deposito']],on='date_file',how='left')

In [ ]:
sftp['Fecha txn'] = pd.to_datetime(sftp['Fecha txn'], format='%Y%m%d').dt.date
sftp['date_file'] = pd.to_datetime(sftp['date_file'], format='%Y%m%d').dt.date
sftp['Fecha_deposito'] = pd.to_datetime(sftp['Fecha_deposito'], format='%Y%m%d').dt.date

In [ ]:
sftp['Net Amount'] = sftp['Importe Bruto']/100
sftp['LAST4'] = sftp['Tarjeta'].str[-4:].astype(str)
sftp['CONCAT'] = sftp['order_id'].astype(str).replace(' ', '',regex=True)+'-'+sftp['Net Amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+sftp['LAST4'].astype(str).replace(' ', '',regex=True)
sftp['order_id'].fillna('NO EXISTE EN SFTP',inplace=True)
sftp.head()

In [ ]:
sftp.shape

In [ ]:
data_status_1 = provision_bnte.merge(sftp[['order_id','date_file', 'Fecha txn', 'Fecha_deposito']],on='order_id',how='left')
data_status_1['date_file'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_1['Fecha txn'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_1['Fecha_deposito'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_1.drop_duplicates(subset ="CONCAT", keep = 'last', inplace = True) 
data_status_1.head()

In [ ]:
data_status_1.shape

In [ ]:
duplicates_status_1 = data_status_1[data_status_1.duplicated(['CONCAT'] , keep=False)]
duplicates_status_1.head()

In [ ]:
duplicates_status_1.shape

In [ ]:
results_status_1 = data_status_1[(data_status_1['Fecha txn'] != 'NO ENCONTRADO EN SFTP')]
results_status_1.head()

In [ ]:
results_status_1.shape

In [ ]:
NA_status_1 = data_status_1[(data_status_1['Fecha txn'] == 'NO ENCONTRADO EN SFTP')]
NA_status_1.head()

In [ ]:
NA_status_1.shape

### INDEX USANDO CLAVE_AUTORIZACION

In [ ]:
sinref = NA_status_1
sinref.drop(['Fecha txn','date_file','Fecha_deposito','CONCAT'],axis=1,inplace=True)
sinref['CONCAT'] = sinref['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'-'+sinref['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+sinref['last4'].astype(str).replace(' ', '',regex=True)
sinref.sort_values(by=('created_at'), ascending=True, inplace=True)
sinref.head()

In [ ]:
sinref.shape

In [ ]:
sftp_clave_autorizacion = pd.concat([sftp_apr_21,sftp_may_21])
# sftp_clave_autorizacion.drop(['Empty','Empty.1','Empty.2','Empty.3','Empty.4'],axis=1,inplace=True)
sftp_clave_autorizacion.rename(columns ={'Ref. Interbancaria':'order_id'}, inplace=True)

In [ ]:
sftp_fecha_deposito_clave_autorizacion = sftp_clave_autorizacion[(sftp_clave_autorizacion['Tipo txn'] == 'BANORTE')]
sftp_fecha_deposito_clave_autorizacion = sftp_fecha_deposito_clave_autorizacion[['date_file', 'Importe Bruto']]
sftp_fecha_deposito_clave_autorizacion.rename(columns ={'Importe Bruto': 'Fecha_deposito'},inplace=True)
sftp_fecha_deposito_clave_autorizacion.head()

In [ ]:
sftp_clave_autorizacion = sftp_clave_autorizacion[(sftp_clave_autorizacion['Tipo txn'] == 'SETTLED')]
sftp_clave_autorizacion = sftp_clave_autorizacion.merge(sftp_fecha_deposito_clave_autorizacion[['date_file', 'Fecha_deposito']],on='date_file',how='left')

In [ ]:
sftp_clave_autorizacion['Fecha txn'] = pd.to_datetime(sftp_clave_autorizacion['Fecha txn'], format='%Y%m%d').dt.date
sftp_clave_autorizacion['date_file'] = pd.to_datetime(sftp_clave_autorizacion['date_file'], format='%Y%m%d').dt.date
sftp_clave_autorizacion['Fecha_deposito'] = pd.to_datetime(sftp_clave_autorizacion['Fecha_deposito'], format='%Y%m%d').dt.date

In [ ]:
sftp_clave_autorizacion['Net Amount'] = sftp_clave_autorizacion['Importe Bruto']/100
sftp_clave_autorizacion['last4'] = sftp_clave_autorizacion['Tarjeta'].str[-4:].astype(str)
sftp_clave_autorizacion['CONCAT'] = sftp_clave_autorizacion['Código de Autorización'].astype(str).replace(' ', '',regex=True)+'-'+sftp_clave_autorizacion['Net Amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+sftp_clave_autorizacion['last4'].astype(str).replace(' ', '',regex=True)
sftp_clave_autorizacion['order_id'].fillna('NO EXISTE EN SFTP',inplace=True)
sftp_clave_autorizacion.head()

In [ ]:
sftp_clave_autorizacion.shape

In [ ]:
results2 = sinref.merge(sftp_clave_autorizacion[['CONCAT','SFTP', 'Fecha txn', 'Fecha_deposito']],on='CONCAT',how='left')
results2['Fecha txn'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
results2['Fecha_deposito'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
results2.drop_duplicates(subset ="CONCAT", keep = 'last', inplace = True)
results2.head()

In [ ]:
results2.shape

In [ ]:
results2_status_1 = results2[(results2['Fecha txn'] != 'NO ENCONTRADO EN SFTP')]
results2_status_1.head()

In [ ]:
results2_status_1.shape

In [ ]:
NA2 = results2[(results2['Fecha txn'] == 'NO ENCONTRADO EN SFTP')]
NA2.head()

In [ ]:
NA2.shape

In [ ]:
duplicates = results2[results2.duplicated(['CONCAT'] , keep=False)]
duplicates.head()

In [ ]:
duplicates.shape

### DATA FINAL

In [ ]:
provision_prosa_final = pd.concat([results_status_1,NA_status_1])
provision_prosa_final.drop(['SFTP'],axis=1,inplace=True)
provision_prosa_final['status'].fillna('RETAINED',inplace=True)
provision_prosa_final['report_date_local'].fillna('NO ENCONTRADO',inplace=True)
provision_prosa_final.head()

In [ ]:
provision_prosa_final.shape

### PIVOT

#### REPORT_DATE_LOCAL

In [ ]:
pivot_bnte_report_date_local = pd.pivot_table(provision_prosa_final,index=['date_transaction','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_bnte_report_date_local.fillna(0,inplace=True)
pivot_bnte_report_date_local.head()

In [ ]:
pivot_bnte_report_date_local.shape

#### FECHA_DEPOSITO

In [ ]:
pivot_bnte_fecha_deposito = pd.pivot_table(provision_prosa_final,index=['date_transaction','clip_reader']
                                              ,columns=['Fecha_deposito'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_bnte_fecha_deposito.fillna(0,inplace=True)
pivot_bnte_fecha_deposito.head()

In [ ]:
pivot_bnte_fecha_deposito.shape

In [ ]:
# writer = pd.ExcelWriter('SF PROVISIONES SEPTIEMBRE 2020.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
# provision_prosa_final.to_excel(writer, sheet_name='SEPTIEMBRE BNTE', index=False)
# pivot_bnte_report_date_local.to_excel(writer, sheet_name='PIVOT BNTE')
# writer.save()

# PROVISIONES AMEX

## DATA EPA

In [ ]:
allfiles = glob.glob(r'G:\My Drive\EPA\EPA files\20**\**\*.txt', recursive=True)

In [ ]:
names_header=["1","2","3","4","5","6","7","8","9","10","A","12","13","14","15","16","17","18","19","order_id","21","22","23","24","25","26","27","28"]

In [ ]:
epa = pd.concat([pd.read_csv(f,names=names_header, sep=',',dtype=object) for f in allfiles])
epa['order_id'] = epa['order_id'].str.strip()
epa.head()

In [ ]:
epa.shape

In [ ]:
epa['6'].unique()

In [ ]:
epa_0 = epa[(epa['6'] == '0')]
epa_1 = epa[(epa['6'] == '1')]
epa_3 = epa[(epa['6'] == '3')]
epa_4 = epa[(epa['6'] == '4')]
epa_5 = epa[(epa['6'] == '5')]
epa_9 = epa[(epa['6'] == '9')]

### EPA 4

In [ ]:
epa_4['8'] = pd.to_datetime(epa_4['8'], format='%Y%m%d').dt.date
epa_4['2'] = pd.to_datetime(epa_4['2'], format='%Y%m%d').dt.date

In [ ]:
epa_4['8'] = pd.to_datetime(epa_4['8'])
epa_4['2'] = pd.to_datetime(epa_4['2'])

In [ ]:
epa_4.head()

In [ ]:
epa_4.shape

In [ ]:
txn_epa = epa_4[['order_id','6','8','A','12','22','23','2','15']]
txn_epa["12"] = txn_epa['12'].astype(float)
txn_epa["22"] = txn_epa['22'].astype(float)
txn_epa["23"] = txn_epa['23'].astype(float)


In [ ]:
txn_epa['total_amount_bruto'] = txn_epa['12']/100
txn_epa['fee_amount_mdr'] = -(txn_epa['total_amount_bruto']*0.0240)
txn_epa['fee_amount_msi'] = txn_epa['22']/100
txn_epa['iva_msi'] = txn_epa['fee_amount_msi']*0.1600
txn_epa['iva_mdr'] = txn_epa['fee_amount_mdr']*0.1600
txn_epa['bin_bank'] = txn_epa['A'].str[:6].astype(str)
txn_epa['last4_bank'] = txn_epa['A'].str[-8:-4].astype(str)

In [ ]:
txn_epa.rename(columns ={'6':'Type','8':'Date_process_roc','12':'Total_amount_charge_roc',
                         '22':'Svc_fee_amount_roc','23':'Svc_fee_amount_accel_roc','2':'roc_deposit_date'},inplace=True)
txn_epa['INDEX_TOTAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['total_amount_bruto'].astype(str).replace(' ', '',regex=True))
# txn_epa['INDEX_PARCIAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['codigo_aprobacion'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['total_amount_bruto'].astype(str).replace(' ', '',regex=True))
txn_epa.fillna('SIN ORDER_ID EN EPA',inplace=True)
txn_epa.head()

In [ ]:
txn_epa.shape

## CRUCE INFORMACION

### DATA FINAL

In [ ]:
provision_amex_1 = sim_amex.merge(txn_epa[['order_id','Date_process_roc','roc_deposit_date']]
                                        ,on='order_id',how='left')
#provision_amex_final.fillna('RETAINED',inplace=True)
provision_amex_1.rename(columns ={'Date_process_roc': 'date_process','roc_deposit_date': 'deposit_date'},inplace=True)
provision_amex_1['date_process'].fillna('NO ENCONTRADO EN EPA',inplace=True)
provision_amex_1['deposit_date'].fillna('NO ENCONTRADO EN EPA',inplace=True)
provision_amex_1.head()

In [ ]:
provision_amex_1.shape

In [ ]:
results_amex_final = provision_amex_1[(provision_amex_1['deposit_date'] != 'NO ENCONTRADO EN EPA')]
results_amex_final.head()

In [ ]:
results_amex_final.shape

In [ ]:
results_amex_final['date_process'] = pd.to_datetime(results_amex_final['date_process'])
results_amex_final['deposit_date'] = pd.to_datetime(results_amex_final['deposit_date'])

In [ ]:
results_amex_final.head()

In [ ]:
results_amex_final.shape

### PIVOT

#### REPORT_DATE_LOCAL

In [ ]:
pivot_amex_report_date_local = pd.pivot_table(results_amex_final,index=['date_transaction','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_amex_report_date_local.fillna(0,inplace=True)
pivot_amex_report_date_local.head()

In [ ]:
pivot_amex_report_date_local.shape

#### FECHA_DEPOSITO

In [ ]:
pivot_amex_fecha_deposito = pd.pivot_table(results_amex_final,index=['date_transaction','clip_reader']
                                              ,columns=['deposit_date'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_amex_fecha_deposito.fillna(0,inplace=True)
pivot_amex_fecha_deposito.head()

In [ ]:
pivot_amex_fecha_deposito.shape

In [ ]:
# book = load_workbook('SF PROVISIONES SEPTIEMBRE 2020.xlsx')
# writer = pd.ExcelWriter('SF PROVISIONES SEPTIEMBRE 2020.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# provision_amex_final.to_excel(writer, sheet_name='SEPTIEMBRE AMEX', index=False)
# writer.save()
# writer.close()

# PROVISIONES OPTBLUE

In [ ]:
provision_optb = provision_amex_1[(provision_amex_1['deposit_date'] == 'NO ENCONTRADO EN EPA')]
provision_optb['order_id'] = provision_optb['order_id'].str.upper() 
provision_optb['CONCAT'] = provision_optb['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_optb['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_optb['last4'].astype(str).replace(' ', '',regex=True)
provision_optb.drop(['date_process','deposit_date'],axis=1,inplace=True)
provision_optb.head()

In [ ]:
provision_optb.shape

In [ ]:
opt = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\PROVISIONES\2021\03. MARZO\SFTP\AMEX PROVISION OPTBLUE MARZO 2021.csv')   
opt.head()


In [ ]:
opt.shape

In [ ]:
opt.columns

### DATA FINAL

In [ ]:
provision_optb_final = provision_optb.merge(opt[['order_id','fecha_pago','fecha_transaccion']]
                                        ,on='order_id',how='left')
#provision_amex_final.fillna('RETAINED',inplace=True)
provision_optb_final.rename(columns ={'Date_process_roc': 'date_process','roc_deposit_date': 'deposit_date'},inplace=True)
provision_optb_final.head()

In [ ]:
provision_optb_final['fecha_pago'] = pd.to_datetime(provision_optb_final['fecha_pago'])
provision_optb_final['fecha_transaccion'] = pd.to_datetime(provision_optb_final['fecha_transaccion'])

In [ ]:
provision_optb_final.head()

In [ ]:
provision_optb_final.shape

In [ ]:
optb_not_found = provision_optb_final[(provision_optb_final['fecha_pago'] == 'NO ENCONTRADO EN EPA')]
optb_not_found.head()

In [ ]:
optb_not_found.shape

### PIVOT 

#### REPORT DATE LOCAL

In [ ]:
pivot_optb_report_date_local = pd.pivot_table(provision_optb_final,index=['date_transaction','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_optb_report_date_local.fillna(0,inplace=True)
pivot_optb_report_date_local.head()

In [ ]:
pivot_optb_report_date_local.shape

#### FECHA DEPOSITO

In [ ]:
pivot_optb_fecha_deposito = pd.pivot_table(provision_optb_final,index=['date_transaction','clip_reader']
                                              ,columns=['fecha_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_optb_fecha_deposito.fillna(0,inplace=True)
pivot_optb_fecha_deposito.head()

In [ ]:
pivot_optb_fecha_deposito.shape

# PROVISIONES ECOMMERCE

In [ ]:
provision_ecommerce = all_provisiones[(all_provisiones['psp'] == 'PAYWORKS-BANORTE')]
provision_ecommerce.rename(columns ={'order_id':'order_id_payworks'}, inplace=True)
provision_ecommerce['CONCAT'] = provision_ecommerce['order_id_payworks'].astype(str).replace(' ', '',regex=True)+'-'+provision_ecommerce['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_ecommerce['last4'].astype(str).replace(' ', '',regex=True)
provision_ecommerce.head()

In [ ]:
provision_ecommerce.shape

In [ ]:
data_status_ecom_1 = provision_ecommerce.merge(sftp[['order_id_payworks','order_id','SFTP', 'Fecha txn', 'Fecha_deposito']],on='order_id_payworks',how='left')
data_status_ecom_1['SFTP'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_ecom_1['Fecha txn'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_ecom_1['Fecha_deposito'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_ecom_1.drop_duplicates(subset ="CONCAT", keep = 'last', inplace = True) 
data_status_ecom_1.head()

In [ ]:
data_status_ecom_1.shape

In [ ]:
duplicates_status_ecom_1 = data_status_ecom_1[data_status_ecom_1.duplicated(['CONCAT'] , keep=False)]
duplicates_status_ecom_1.head()

In [ ]:
duplicates_status_ecom_1.shape

In [ ]:
results_status_ecom_1 = data_status_ecom_1[(data_status_ecom_1['Fecha txn'] != 'NO ENCONTRADO EN SFTP')]
results_status_ecom_1.head()

In [ ]:
results_status_ecom_1.shape

In [ ]:
NA_status_ecom_1 = data_status_ecom_1[(data_status_ecom_1['Fecha txn'] == 'NO ENCONTRADO EN SFTP')]
NA_status_ecom_1.head()

In [ ]:
NA_status_ecom_1.shape

### INDEX USANDO CLAVE_AUTORIZACION

In [ ]:
sinref_ecom = NA_status_ecom_1
sinref_ecom.drop(['Fecha txn','SFTP','Fecha_deposito','CONCAT'],axis=1,inplace=True)
sinref_ecom['CONCAT'] = sinref_ecom['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'-'+sinref_ecom['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+sinref_ecom['last4'].astype(str).replace(' ', '',regex=True)
sinref_ecom.sort_values(by=('created_at'), ascending=True, inplace=True)
sinref_ecom.head()

In [ ]:
sinref_ecom.shape

In [ ]:
sftp_clave_autorizacion_ecom = pd.concat([df1,df2,df3,df4,df5])
# sftp_clave_autorizacion.drop(['Empty','Empty.1','Empty.2','Empty.3','Empty.4'],axis=1,inplace=True)
sftp_clave_autorizacion_ecom.rename(columns ={'Ref. Interbancaria':'order_id'}, inplace=True)

In [ ]:
sftp_fecha_deposito_clave_autorizacion_ecom = sftp_clave_autorizacion_ecom[(sftp_clave_autorizacion_ecom['Tipo txn'] == 'BANORTE')]
sftp_fecha_deposito_clave_autorizacion_ecom = sftp_fecha_deposito_clave_autorizacion_ecom[['SFTP', 'Importe Bruto']]
sftp_fecha_deposito_clave_autorizacion_ecom.rename(columns ={'Importe Bruto': 'Fecha_deposito'},inplace=True)
sftp_fecha_deposito_clave_autorizacion_ecom.head()

In [ ]:
sftp_clave_autorizacion_ecom = sftp_clave_autorizacion_ecom[(sftp_clave_autorizacion_ecom['Tipo txn'] == 'SETTLED')]
sftp_clave_autorizacion_ecom = sftp_clave_autorizacion_ecom.merge(sftp_fecha_deposito_clave_autorizacion_ecom[['SFTP', 'Fecha_deposito']],on='SFTP',how='left')

In [ ]:
sftp_clave_autorizacion_ecom['Fecha txn'] = pd.to_datetime(sftp_clave_autorizacion_ecom['Fecha txn'], format='%Y%m%d').dt.date
sftp_clave_autorizacion_ecom['SFTP'] = pd.to_datetime(sftp_clave_autorizacion_ecom['SFTP'], format='%Y%m%d').dt.date
sftp_clave_autorizacion_ecom['Fecha_deposito'] = pd.to_datetime(sftp_clave_autorizacion_ecom['Fecha_deposito'], format='%Y%m%d').dt.date

In [ ]:
sftp_clave_autorizacion_ecom['Net Amount'] = sftp_clave_autorizacion_ecom['Importe Bruto']/100
sftp_clave_autorizacion_ecom['last4'] = sftp_clave_autorizacion_ecom['Tarjeta'].str[-4:].astype(str)
sftp_clave_autorizacion_ecom['CONCAT'] = sftp_clave_autorizacion_ecom['Código de Autorización'].astype(str).replace(' ', '',regex=True)+'-'+sftp_clave_autorizacion_ecom['Net Amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+sftp_clave_autorizacion_ecom['last4'].astype(str).replace(' ', '',regex=True)
sftp_clave_autorizacion_ecom['order_id'].fillna('NO EXISTE EN SFTP',inplace=True)
sftp_clave_autorizacion_ecom.head()

In [ ]:
sftp_clave_autorizacion_ecom.shape

In [ ]:
results2_ecom = sinref_ecom.merge(sftp_clave_autorizacion_ecom[['CONCAT','SFTP', 'Fecha txn', 'Fecha_deposito']],on='CONCAT',how='left')
results2_ecom['Fecha txn'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
results2_ecom['Fecha_deposito'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
results2_ecom.drop_duplicates(subset ="CONCAT", keep = 'last', inplace = True)
results2_ecom.head()

In [ ]:
results2_ecom.shape

In [ ]:
results2_status_1_ecom = results2_ecom[(results2_ecom['Fecha txn'] != 'NO ENCONTRADO EN SFTP')]
results2_status_1_ecom.head()

In [ ]:
results2_status_1_ecom.shape

In [ ]:
NA2_ecom = results2_ecom[(results2_ecom['Fecha txn'] == 'NO ENCONTRADO EN SFTP')]
NA2_ecom.head()

In [ ]:
NA2_ecom.shape

In [ ]:
duplicates_ecom = results2_ecom[results2_ecom.duplicated(['CONCAT'] , keep=False)]
duplicates_ecom.head()

In [ ]:
duplicates_ecom.shape

### DATA FINAL

In [ ]:
provision_xpay_final = pd.concat([results_status_ecom_1,results2_status_1_ecom,NA2_ecom])
provision_xpay_final.drop(['SFTP'],axis=1,inplace=True)
provision_xpay_final['status'].fillna('RETAINED',inplace=True)
provision_xpay_final['report_date_local'].fillna('NO ENCONTRADO',inplace=True)
provision_xpay_final.head()

In [ ]:
provision_xpay_final.shape

### PIVOT

#### REPORT_DATE_LOCAL

In [ ]:
pivot_xpay_report_date_local = pd.pivot_table(provision_xpay_final,index=['date_transaction','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_xpay_report_date_local.fillna(0,inplace=True)
pivot_xpay_report_date_local.head()

In [ ]:
pivot_xpay_report_date_local.shape

#### FECHA_DEPOSITO

In [ ]:
pivot_xpay_fecha_deposito = pd.pivot_table(provision_xpay_final,index=['date_transaction','clip_reader']
                                              ,columns=['Fecha_deposito'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_xpay_fecha_deposito.fillna(0,inplace=True)
pivot_xpay_fecha_deposito.head()

In [ ]:
pivot_xpay_fecha_deposito.shape

In [ ]:
# book = load_workbook('PROVISIONES SEPTIEMBRE 2020.xlsx')
# writer = pd.ExcelWriter('PROVISIONES SEPTIEMBRE 2020.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# provision_ecommerce_total.to_excel(writer, sheet_name='SEPTIEMBRE XPAY', index=False)
# writer.save()
# writer.close()

# PROVISIONES MIT-3DS

In [ ]:
provision_mit = all_provisiones[(all_provisiones['psp'] == 'MIT-3DS')]
provision_mit['CONCAT'] = provision_mit['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_mit['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_mit['last4'].astype(str).replace(' ', '',regex=True)
provision_mit.head()

In [ ]:
provision_mit.shape

In [ ]:
data_status_mit = provision_mit.merge(sftp[['order_id','SFTP', 'Fecha txn', 'Fecha_deposito']],on='order_id',how='left')
data_status_mit['SFTP'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_mit['Fecha txn'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_mit['Fecha_deposito'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
data_status_mit.drop_duplicates(subset ="CONCAT", keep = 'last', inplace = True) 
data_status_mit.head()

In [ ]:
data_status_mit.shape

In [ ]:
duplicates_status_mit = data_status_mit[data_status_mit.duplicated(['CONCAT'] , keep=False)]
duplicates_status_mit.head()

In [ ]:
duplicates_status_mit.shape

In [ ]:
results_status_mit = data_status_mit[(data_status_mit['Fecha txn'] != 'NO ENCONTRADO EN SFTP')]
results_status_mit.head()

In [ ]:
results_status_mit.shape

In [ ]:
NA_status_mit = data_status_mit[(data_status_mit['Fecha txn'] == 'NO ENCONTRADO EN SFTP')]
NA_status_mit.head()

In [ ]:
NA_status_mit.shape

### INDEX USANDO CLAVE AUTORIZACION

In [ ]:
sinref_mit = NA_status_mit
sinref_mit.drop(['Fecha txn','SFTP','Fecha_deposito','CONCAT'],axis=1,inplace=True)
sinref_mit['CONCAT'] = sinref_mit['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'-'+sinref_mit['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+sinref_mit['last4'].astype(str).replace(' ', '',regex=True)
sinref_mit.sort_values(by=('created_at'), ascending=True, inplace=True)
sinref_mit.head()

In [ ]:
sinref_mit.shape

In [ ]:
sftp_clave_autorizacion_mit = pd.concat([df1,df2,df3,df4,df5])
# sftp_clave_autorizacion.drop(['Empty','Empty.1','Empty.2','Empty.3','Empty.4'],axis=1,inplace=True)
sftp_clave_autorizacion_mit.rename(columns ={'Ref. Interbancaria':'order_id'}, inplace=True)

In [ ]:
sftp_fecha_deposito_clave_autorizacion_mit = sftp_clave_autorizacion_mit[(sftp_clave_autorizacion_mit['Tipo txn'] == 'BANORTE')]
sftp_fecha_deposito_clave_autorizacion_mit = sftp_fecha_deposito_clave_autorizacion_mit[['SFTP', 'Importe Bruto']]
sftp_fecha_deposito_clave_autorizacion_mit.rename(columns ={'Importe Bruto': 'Fecha_deposito'},inplace=True)
sftp_fecha_deposito_clave_autorizacion_mit.head()

In [ ]:
sftp_clave_autorizacion_mit = sftp_clave_autorizacion_mit[(sftp_clave_autorizacion_mit['Tipo txn'] == 'SETTLED')]
sftp_clave_autorizacion_mit = sftp_clave_autorizacion_mit.merge(sftp_fecha_deposito_clave_autorizacion_mit[['SFTP', 'Fecha_deposito']],on='SFTP',how='left')

In [ ]:
sftp_clave_autorizacion_mit['Fecha txn'] = pd.to_datetime(sftp_clave_autorizacion_mit['Fecha txn'], format='%Y%m%d').dt.date
sftp_clave_autorizacion_mit['SFTP'] = pd.to_datetime(sftp_clave_autorizacion_mit['SFTP'], format='%Y%m%d').dt.date
sftp_clave_autorizacion_mit['Fecha_deposito'] = pd.to_datetime(sftp_clave_autorizacion_mit['Fecha_deposito'], format='%Y%m%d').dt.date

In [ ]:
sftp_clave_autorizacion_mit['Net Amount'] = sftp_clave_autorizacion_mit['Importe Bruto']/100
sftp_clave_autorizacion_mit['last4'] = sftp_clave_autorizacion_mit['Tarjeta'].str[-4:].astype(str)
sftp_clave_autorizacion_mit['CONCAT'] = sftp_clave_autorizacion_mit['Código de Autorización'].astype(str).replace(' ', '',regex=True)+'-'+sftp_clave_autorizacion_mit['Net Amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+sftp_clave_autorizacion_mit['last4'].astype(str).replace(' ', '',regex=True)
sftp_clave_autorizacion_mit['order_id'].fillna('NO EXISTE EN SFTP',inplace=True)
sftp_clave_autorizacion_mit.head()

In [ ]:
sftp_clave_autorizacion_mit.shape

In [ ]:
results2_mit = sinref_mit.merge(sftp_clave_autorizacion_mit[['CONCAT','SFTP', 'Fecha txn', 'Fecha_deposito']],on='CONCAT',how='left')
results2_mit['Fecha txn'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
results2_mit['Fecha_deposito'].fillna('NO ENCONTRADO EN SFTP',inplace=True)
results2_mit.drop_duplicates(subset ="CONCAT", keep = 'last', inplace = True)
results2_mit.head()

In [ ]:
results2_mit.shape

In [ ]:
results2_status_1_mit = results2_mit[(results2_mit['Fecha txn'] != 'NO ENCONTRADO EN SFTP')]
results2_status_1_mit.head()

In [ ]:
results2_status_1_mit.shape

In [ ]:
NA2_mit = results2_mit[(results2_mit['Fecha txn'] == 'NO ENCONTRADO EN SFTP')]
NA2_mit.head()

In [ ]:
NA2_mit.shape

In [ ]:
duplicates_mit = results2_mit[results2_mit.duplicated(['CONCAT'] , keep=False)]
duplicates_mit.head()

In [ ]:
duplicates_mit.shape

### DATA FINAL

In [ ]:
provision_mit_final = pd.concat([results_status_mit,results2_status_1_mit,NA2_mit])
provision_mit_final['status'].fillna('RETAINED',inplace=True)
provision_mit_final['report_date_local'].fillna('NO ENCONTRADO',inplace=True)
provision_mit_final.head()

In [ ]:
provision_mit_final.shape

### PIVOT 

#### REPORT_DATE_LOCAL

In [ ]:
pivot_mit_report_date_local = pd.pivot_table(provision_mit_final,index=['date_transaction','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_mit_report_date_local.fillna(0,inplace=True)
pivot_mit_report_date_local.head()

In [ ]:
pivot_mit_report_date_local.shape

#### FECHA_DEPOSITO

In [ ]:
pivot_mit_fecha_deposito = pd.pivot_table(provision_mit_final,index=['date_transaction','clip_reader']
                                              ,columns=['Fecha_deposito'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_mit_fecha_deposito.fillna(0,inplace=True)
pivot_mit_fecha_deposito.head()

In [ ]:
pivot_mit_fecha_deposito.shape

In [ ]:
# book = load_workbook('PROVISIONES SEPTIEMBRE 2020.xlsx')
# writer = pd.ExcelWriter('PROVISIONES SEPTIEMBRE 2020.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# provision_mit_total.to_excel(writer, sheet_name='SEPTIEMBRE OPENPAY', index=False)
# writer.save()
# writer.close()

# PROVISIONES OPEN PAY 

In [ ]:
provision_openpay = all_provisiones[(all_provisiones['psp'] == 'OPENPAY')]
provision_openpay['CONCAT'] = provision_openpay['order_id'].astype(str).replace(' ', '',regex=True)+'-'+provision_openpay['amount'].apply(np.floor).astype(str).replace(' ', '',regex=True)+'-'+provision_openpay['last4'].astype(str).replace(' ', '',regex=True)
provision_openpay.head()

In [ ]:
provision_openpay.shape

### DATA FINAL

In [ ]:
provision_openpay_final = pd.concat([provision_openpay])
provision_openpay_final.fillna('RETAINED',inplace=True)
provision_openpay_final.head()

In [ ]:
provision_openpay_final.shape

### PIVOT 

#### REPORT_DATE_LOCAL

In [ ]:
pivot_openpay_report_date_local = pd.pivot_table(provision_openpay_final,index=['date_transaction','clip_reader']
                                              ,columns=['report_date_local'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index(1)
pivot_openpay_report_date_local.fillna(0,inplace=True)
pivot_openpay_report_date_local.head()

In [ ]:
pivot_openpay_report_date_local.shape

In [ ]:
# book = load_workbook('PROVISIONES SEPTIEMBRE 2020.xlsx')
# writer = pd.ExcelWriter('PROVISIONES SEPTIEMBRE 2020.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# provision_openpay_final.to_excel(writer, sheet_name='SEPTIEMBRE OPENPAY', index=False)
# writer.save()
# writer.close()

# EXCEL FINAL 

In [ ]:
# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('DATA PROVISIONES MARZO 2021.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})   
workbook=writer.book

# BNTE
worksheet=workbook.add_worksheet('MARZO BNTE')
writer.sheets['MARZO BNTE'] = worksheet
provision_prosa_final.to_excel(writer,sheet_name='MARZO BNTE',startrow=0 , startcol=0, index=False)   
pivot_bnte_report_date_local.to_excel(writer,sheet_name='MARZO BNTE',startrow=1, startcol=21)
pivot_bnte_fecha_deposito.to_excel(writer,sheet_name='MARZO BNTE',startrow=12, startcol=21)

# AMEX
worksheet=workbook.add_worksheet('MARZO AMEX')
writer.sheets['MARZO AMEX'] = worksheet
results_amex_final.to_excel(writer,sheet_name='MARZO AMEX',startrow=0 , startcol=0, index=False)  
pivot_amex_report_date_local.to_excel(writer,sheet_name='MARZO AMEX',startrow=1, startcol=21)
pivot_amex_fecha_deposito.to_excel(writer,sheet_name='MARZO AMEX',startrow=12, startcol=21)

#OPTBLUE
worksheet=workbook.add_worksheet('MARZO OPTBLUE')
writer.sheets['MARZO OPTBLUE'] = worksheet
provision_optb_final.to_excel(writer,sheet_name='MARZO OPTBLUE',startrow=0 , startcol=0, index=False)  
pivot_optb_report_date_local.to_excel(writer,sheet_name='MARZO OPTBLUE',startrow=1, startcol=21)
pivot_optb_fecha_deposito.to_excel(writer,sheet_name='MARZO OPTBLUE',startrow=12, startcol=21)

# XPAY
worksheet=workbook.add_worksheet('MARZO XPAY')
writer.sheets['MARZO XPAY'] = worksheet
provision_xpay_final.to_excel(writer,sheet_name='MARZO XPAY',startrow=0 , startcol=0, index=False)  
pivot_xpay_report_date_local.to_excel(writer,sheet_name='MARZO XPAY',startrow=1, startcol=22)
pivot_xpay_fecha_deposito.to_excel(writer,sheet_name='MARZO XPAY',startrow=12, startcol=22)

# MIT
worksheet=workbook.add_worksheet('MARZO MIT')
writer.sheets['MARZO MIT'] = worksheet
provision_mit_final.to_excel(writer,sheet_name='MARZO MIT',startrow=0 , startcol=0, index=False)  
pivot_mit_report_date_local.to_excel(writer,sheet_name='MARZO MIT',startrow=1, startcol=22)
pivot_mit_fecha_deposito.to_excel(writer,sheet_name='MARZO MIT',startrow=12, startcol=22)

# OPENPAY
worksheet=workbook.add_worksheet('MARZO OPENPAY')
writer.sheets['MARZO OPENPAY'] = worksheet
provision_openpay_final.to_excel(writer,sheet_name='MARZO OPENPAY',startrow=0 , startcol=0, index=False)  
pivot_openpay_report_date_local.to_excel(writer,sheet_name='MARZO OPENPAY',startrow=1, startcol=21)

writer.save()
writer.close()